In [ ]:
#import all library required

import numpy as np
import pandas as pd
import tensorflow as tf
from pathlib import Path
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

mpl.style.use("seaborn-darkgrid")

In [ ]:
# switch to GPU

In [ ]:
#import all keras library component 
from tensorflow.keras.layers import (
    Input,
    Conv2D, 
    MaxPool2D, 
    Dense, 
    BatchNormalization, 
    ReLU, 
    Dropout, 
    Flatten,
    Dropout,
    Concatenate,
    GlobalAvgPool2D
)

from tensorflow.keras.regularizers import L2

In [ ]:
#read Dataset
df_train=pd.read_csv("../input/facial-keypoints-detection/training.zip")

In [ ]:
df_train.head(1)
#dataset consist of 30 feature (x and y coordinate ) and image array

In [ ]:
df_train.describe()

In [ ]:
# define feature and target columns
# 
feature_col,target_col="Image",list(df_train.drop("Image",axis=1).columns)

In [ ]:
# replace NaN with mean  
# if found Nan replace
df_train[target_col] = df_train[target_col].fillna(df_train[target_col].mean())
df_train.info()

In [ ]:
target_col # total 30 target (x,y coordinate)

In [ ]:
# define width and height of image
IMG_WIDTH=96
IMG_HEIGHT=96
IMG_CANNELS=1 # grayscale image

## split image and label  #image
images=np.array(df_train[feature_col].str.split().tolist(),dtype="float").reshape(-1,IMG_HEIGHT,IMG_WIDTH,IMG_CANNELS)
labels=df_train[target_col].to_numpy()

In [ ]:
# scale image from 0 -255 to 0-1 for better training
scaled_images=images/255.0

In [ ]:
labels

In [ ]:
# let's make a function to show training dataset visual
# 
def show_examples(images, landmarks):
  # make a figure consist of 16 images
    fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(16, 16))
    # looping through each image
    for img, marks, ax in zip(images, landmarks, axes.ravel()):
        # Keypoints
        # every odd values are x coordinate
        # every even values are y coodinate
        x_points = marks[:: 2]
        y_points = marks[1::2]
        
        # display on figure 
        ax.imshow(img.squeeze(), cmap='gray')
        ax.scatter(x_points, y_points, s=10, color='red')
    
    plt.show()
    

idx = np.random.choice(16, 16)
show_examples(images[idx], labels[idx])

In [ ]:
# spliting training dataset for training and validation
train_image,valid_images,train_labels,valid_labels=train_test_split(scaled_images,labels,test_size=0.1,random_state=7)
# 10 % of data is for validation 
# 90 % of data is for traiing 
# shuffle dataset before spliting with randam state of 7


In [ ]:
valid_labels

In [ ]:
# Let's start with model
# We will create CNN model that uses Inception Architecture
def inception_module(inputs,f1,f2):
    # defining convolution 2d layer 
    x1=Conv2D(f1,3,padding="same")(inputs)
    # batchnormalize the x1
    x1=BatchNormalization()(x1)
    x1=ReLU()(x1)
    
    # it was f1 change it to f2 
    x2=Conv2D(f2,5,padding="same")(inputs)
    # batchnormalize the x1
    x2=BatchNormalization()(x2)
    x2=ReLU()(x2)
    # combine x1 and x2
    return Concatenate()([x1,x2])
    

In [ ]:
def build_model():
    inputs = Input((96, 96, 1))

    x = inception_module(inputs, 64,  32)
    x = MaxPool2D()(x)
    
    x = inception_module(x, 64,  32)
    x = MaxPool2D()(x)
    
    x = inception_module(x, 128, 32)
    x = MaxPool2D()(x)
    
    x = inception_module(x, 128, 32)
    x = MaxPool2D()(x)
    
    x = inception_module(x, 256, 64)
    x = MaxPool2D()(x)
    
    x = Flatten()(x)
    x = Dense(1024, kernel_regularizer=L2(l2=0.05))(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Dense(512, kernel_regularizer=L2(l2=0.02))(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Dense(128, kernel_regularizer=L2(l2=0.01))(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Dense(30)(x)
    
    model = tf.keras.Model(inputs, outputs=x)
    return model

model = build_model()
model.summary()

In [ ]:
#compile model 
# you can change this value to get better accuracy
model.compile(optimizer="adam",loss="mean_squared_error",metrics=["mae"])

In [ ]:
# creating a chechpoint to save model at best accuarcy

ckp_path="trained_model/model"
model_checkpoint=tf.keras.callbacks.ModelCheckpoint(filepath=ckp_path,
                                                   monitor="val_mae",
                                                   mode="auto",
                                                   save_best_only=True,
                                                   save_weights_only=True)

In [ ]:
# create a lr reducer which decrease learning rate when accuarcy does not increase
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(factor=0.9,monitor="val_mae",
                                             mode="auto",cooldown=0,
                                             patience=5,verbose=1,min_lr=1e-5)
# patience : wait till 5 epoch
# verbose : show accuracy every 1 epoch
# min_lr=minimum learning rate
#

In [ ]:
train_labels

In [ ]:
# Training model
EPOCHS=300
BATCH_SIZE=256

history=model.fit(train_image,
                 train_labels,
                 validation_data=(valid_images,valid_labels),
                 batch_size=BATCH_SIZE,
                 epochs=EPOCHS,
                 callbacks=[model_checkpoint,reduce_lr]
                 )
# found the error
# we are getting NaN value in training because there is NaN in train set

In [ ]:
# now  training is working properly

In [ ]:
# I will see you when training is finish

In [ ]:
#now the training is finished load best model from check point
model.load_weights(ckp_path)


In [ ]:
# now save the model in tflite formate
#converter that convert tf to tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
# for testing if model prediction is correct or not 
# use test dataset
df_test=pd.read_csv("../input/facial-keypoints-detection/test.zip")
df_test.head(1)

In [ ]:
test_images=np.array(df_test["Image"].str.split().tolist(),dtype="float").reshape(-1,96,96,1)

In [ ]:
scale_test_images=test_images/255.0

In [ ]:
prediction_val=model.predict(scale_test_images,batch_size=BATCH_SIZE)

In [ ]:
# draw figure using matplot
idx=np.random.choice(16,16)
show_examples(test_images[idx],prediction_val[idx])

In [ ]:
# perfect 
# now download model 
# or you can save version for future use
# download model after creating save is done 


In [ ]:
#Link of model and training will be in description 
# you can copy code from that link 
#

In [ ]:
# let's make a function to show training dataset visual
# 
def show_examples(images, landmarks):
  # make a figure consist of 16 images
    fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(16, 16))
    # looping through each image
    for img, marks, ax in zip(images, landmarks, axes.ravel()):
        # Keypoints
        # every odd values are x coordinate
        # every even values are y coodinate
        x_points = marks[:: 2]
        y_points = marks[1::2]
        
        # display on figure 
        ax.imshow(img.squeeze(), cmap='gray')
    #    ax.scatter(x_points, y_points, s=10, color='red')
    
    plt.show()
    

idx = np.random.choice(16, 16)
show_examples(images[idx], labels[idx])